<a href="https://colab.research.google.com/github/Cameron-IT/huggungface/blob/main/Another_copy_of_%5BTxGemma%5DQuickstart_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

~~~
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
~~~

# Quick start with Hugging Face

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/TxGemma/[TxGemma]Quickstart_with_Hugging_Face.ipynb">
      <img alt="Google Colab logo" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="32px"><br> Run in Google Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-gemini/gemma-cookbook/blob/main/TxGemma/%5BTxGemma%5DQuickstart_with_Hugging_Face.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://huggingface.co/collections/google/txgemma-release-67dd92e931c857d15e4d1e87">
      <img alt="HuggingFace logo" src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" width="32px"><br> View on HuggingFace
    </a>
  </td>
</tr></tbody></table>

This notebook provides a basic demo of using TxGemma, a collection of large language models built upon Gemma 2, that generates predictions, classifications or text based on therapeutic related data. It contains standalone usage examples for both:

- Predictive tasks, which expect a narrow form of prompting
- Conversational use (for TxGemma-Chat model variants), which is more flexible and includes multi-turn interactions

Learn more about the model at [this page](https://developers.google.com/health-ai-developer-foundations/txgemma).

## Setup

To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the TxGemma model. Choose an appropriate runtime when starting your Colab session.

You can try out TxGemma 2B or 9B* for free using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

*To run the demo with TxGemma 9B on a T4 GPU, use int8 quantization to reduce memory usage and speed up inference. Note that the performance of quantized versions has not been evaluated.

### Get access to TxGemma

Before you get started, make sure that you have access to TxGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [TxGemma model page](https://huggingface.co/google/txgemma-2b-predict) and accept the usage conditions.

### Configure your HF token

Generate a Hugging Face `read` access token by clicking [here](https://huggingface.co/settings/tokens) and add your access token to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.

In [1]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies

In [2]:
! pip install --upgrade --quiet accelerate bitsandbytes huggingface_hub transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Load model from Hugging Face Hub

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_VARIANT = "9b-predict"  # @param ["2b-predict", "9b-chat", "9b-predict", "27b-chat", "27b-predict"]

model_id = f"google/txgemma-{MODEL_VARIANT}"

if MODEL_VARIANT == "2b-predict":
    additional_args = {}
else:
    additional_args = {
        "quantization_config": BitsAndBytesConfig(load_in_8bit=True)
    }

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    **additional_args,
)

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

After loading, you can directly use the model and tokenizer, which gives you complete control over the inference process, including tokenization and postprocessing of outputs.

Alternatively, you can use the [`pipeline`](https://www.google.com/url?q=https%3A%2F%2Fhuggingface.co%2Fdocs%2Ftransformers%2Fen%2Fmain_classes%2Fpipelines) API, which provides a simple way to use the model for inference while abstracting away complex details. Here, instantiate a text generation pipeline using the loaded model:

In [4]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


The following sections include standalone examples demonstating how to use the model both directly and with the `pipeline` API. In practice, you should select the method that is best suited for your use case.

## Format prompts for therapeutic tasks

The following sections in this notebook demonstrate prompting TxGemma for therapeutic development tasks from [Therapeutics Data Commons](https://tdcommons.ai/) (TDC).

For these predictive tasks, prompts should be formatted according to the TDC structure, including:

- **Instruction:** Briefly describes the task.

- **Context:** Provides 2-3 sentences of relevant biochemical background, derived from TDC descriptions and literature.

- **Question:** Queries a specific therapeutic property, incorporating textual representations of therapeutics and/or targets as inputs.

  - Inputs can include SMILES strings, amino acid sequences, nucleotide sequences, and natural language text.

  - Optional few-shot examples can be provided.

### Load prompt template

First, load a JSON file that contains the prompt format for various TDC tasks.

In [5]:
import json
from huggingface_hub import hf_hub_download

tdc_prompts_filepath = hf_hub_download(
    repo_id=model_id,
    filename="tdc_prompts.json",
)

with open(tdc_prompts_filepath, "r") as f:
    tdc_prompts_json = json.load(f)

tdc_prompts.json:   0%|          | 0.00/768k [00:00<?, ?B/s]

In [6]:
drugs = {"rivipansel": "C[C@H]1[C@H]([C@H]([C@@H]([C@@H](O1)O[C@@H]2[C@H](C[C@H](C[C@H]2O[C@H]3[C@@H]([C@H]([C@H]([C@H](O3)CO)O)O[C@@H](CC4CCCCC4)C(=O)O)OC(=O)C5=CC=CC=C5)C(=O)NCCNC(=O)COCCOCC(=O)NC6=C7C(=CC(=C6)S(=O)(=O)O)C=C(C=C7S(=O)(=O)O)S(=O)(=O)O)NC(=O)C8=CC(=O)NC(=O)N8)O)O)O",
         "senicapoc": "C1=CC=C(C=C1)C(C2=CC=C(C=C2)F)(C3=CC=C(C=C3)F)C(=O)N",
         "aspirin": "CC(=O)OC1=CC=CC=C1C(=O)O"
         }

diseases = ["sickle cell disease", "Alzheimer's Disease", "glioblastoma", "Amyotrophic lateral sclerosis", "myocardial infarction"]

proteins = {"E-selectin": "MIASQFLSALTLVLLIKESGAWSYNTSTEAMTYDEASAYCQQRYTHLVAIQNKEEIEYLNSILSYSPSYYWIGIRKVNNVWVWVGTQKPLTEEAKNWAPGEPNNRQKDEDCVEIYIKREKDVGMWNDERCSKKKLALCYTAACTNTSCSGHGECVETINNYTCKCDPGFSGLKCEQIVNCTALESPEHGSLVCSHPLGNFSYNSSCSISCDRGYLPSSMETMQCMSSGEWSAPIPACNVVECDAVTNPANGFVECFQNPGSFPWNTTCTFDCEEGFELMGAQSLQCTSSGNWDNEKPTCKAVTCRAVRQPQNGSVRCSHSPAGEFTFKSSCNFTCEEGFMLQGPAQVECTTQGQWTQQIPVCEAFQCTALSNPERGYMNCLPSASGSFRYGSSCEFSCEQGFVLKGSKRLQCGPTGEWDNEKPTCEAVRCDAVHQPPKGLVRCAHSPIGEFTYKSSCAFSCEEGFELHGSTQLECTSQGQWTEEVPSCQVVKCSSLAVPGKINMSCSGEPVFGTVCKFACPEGWTLNGSAARTCGATGHWSGLLPTCEAPTESNIPLVAGLSAAGLSLLTLAPFLLWLRKCLRKAKKFVPASSCQSLESDGSYQKPSYIL",
            "P-selectin": "MANCQIAILYQRFQRVVFGISQLLCFSALISELTNQKEVAAWTYHYSTKAYSWNISRKYCQNRYTDLVAIQNKNEIDYLNKVLPYYSSYYWIGIRKNNKTWTWVGTKKALTNEAENWADNEPNNKRNNEDCVEIYIKSPSAPGKWNDEHCLKKKHALCYTASCQDMSCSKQGECLETIGNYTCSCYPGFYGPECEYVRECGELELPQHVLMNCSHPLGNFSFNSQCSFHCTDGYQVNGPSKLECLASGIWTNKPPQCLAAQCPPLKIPERGNMTCLHSAKAFQHQSSCSFSCEEGFALVGPEVVQCTASGVWTAPAPVCKAVQCQHLEAPSEGTMDCVHPLTAFAYGSSCKFECQPGYRVRGLDMLRCIDSGHWSAPLPTCEAISCEPLESPVHGSMDCSPSLRAFQYDTNCSFRCAEGFMLRGADIVRCDNLGQWTAPAPVCQALQCQDLPVPNEARVNCSHPFGAFRYQSVCSFTCNEGLLLVGASVLQCLATGNWNSVPPECQAIPCTPLLSPQNGTMTCVQPLGSSSYKSTCQFICDEGYSLSGPERLDCTRSGRWTDSPPMCEAIKCPELFAPEQGSLDCSDTRGEFNVGSTCHFSCDNGFKLEGPNNVECTTSGRWSATPPTCKGIASLPTPGVQCPALTTPGQGTMYCRHHPGTFGFNTTCYFGCNAGFTLIGDSTLSCRPSGQWTAVTPACRAVKCSELHVNKPIAMNCSNLWGNFSYGSICSFHCLEGQLLNGSAQTACQENGHWSTTVPTCQAGPLTIQEALTYFGGAVASTIGLIMGGTLLALLRKRFRQKDDGKCPLNPHSHLGTYGVFTNAAFDPSP",
            "Intermediate conductance calcium-activated potassium channel protein 4": "MGGDLVLGLGALRRRKRLLEQEKSLAGWALVLAGTGIGLMVLHAEMLWFGGCSWALYLFLVKCTISISTFLLLCLIVAFHAKEVQLFMTDNGLRDWRVALTGRQAAQIVLELVVCGLHPAPVRGPPCVQDLGAPLTSPQPWPGFLGQGEALLSLAMLLRLYLVPRAVLLRSGVLLNASYRSIGALNQVRFRHWFVAKLYMNTHPGRLLLGLTLGLWLTTAWVLSVAERQAVNATGHLSDTLWLIPITFLTIGYGDVVPGTMWGKIVCLCTGVMGVCCTALLVAVVARKLEFNKAEKHVHNFMMDIQYTKEMKESAARVLQEAWMFYKHTRRKESHAARRHQRKLLAAINAFRQVRLKHRKLREQVNSMVDISKMHMILYDLQQNLSSSHRALEKQIDTLAGKLDALTELLSTALGPRQLPEPSQQSK"
            }

prompts = ["BBB_Martins", "BindingDB_kd", "BindingDB_ic50", "BindingDB_ki", "BindingDB_Patent"]

In [7]:
for p in prompts:
  print(p, "\n", tdc_prompts_json[p], "\n")

BBB_Martins 
 Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: {Drug SMILES}
Answer: 

BindingDB_kd 
 Instructions: Answer the following question about drug target interactions.
Context: Drug-target binding is the physical interaction between a drug and a specific biological molecule, such as a protein or enzyme. This interaction is essential for the drug to exert its pharmacological effect. The strength of the drug-target binding is determined by the binding affinity, which is a measure of how tightly the drug binds to the tar

In [8]:
for t in tdc_prompts_json.keys():
  print(t)

Pgp_Broccatelli
Bioavailability_Ma
BBB_Martins
CYP2D6_Veith
CYP3A4_Veith
CYP2C9_Veith
CYP2D6_Substrate_CarbonMangels
CYP3A4_Substrate_CarbonMangels
CYP2C9_Substrate_CarbonMangels
hERG
AMES
DILI
HIA_Hou
Tox21_NR_AhR
Tox21_NR_Aromatase
Tox21_NR_AR
Tox21_NR_AR_LBD
Tox21_NR_ER
Tox21_NR_ER_LBD
Tox21_NR_PPAR_gamma
Tox21_SR_ARE
Tox21_SR_ATAD5
Tox21_SR_HSE
Tox21_SR_MMP
Tox21_SR_p53
PAMPA_NCATS
CYP2C19_Veith
CYP1A2_Veith
Skin_Reaction
Carcinogens_Lagunin
SARSCoV2_Vitro_Touret
SARSCOV2_3CLPro_Diamond
HIV
ClinTox
orexin1_receptor_butkiewicz
m1_muscarinic_receptor_agonists_butkiewicz
m1_muscarinic_receptor_antagonists_butkiewicz
potassium_ion_channel_kir2.1_butkiewicz
kcnq2_potassium_channel_butkiewicz
cav3_t_type_calcium_channels_butkiewicz
choline_transporter_butkiewicz
serine_threonine_kinase_33_butkiewicz
tyrosyl_dna_phosphodiesterase_butkiewicz
herg_central
hERG_Karim
MHC1_IEDB_IMGT_Nielsen
MHC2_IEDB_Jensen
weber
HuRI
SAbDab_Chen
miRTarBase
ToxCast_ACEA_T47D_80hr_Negative
ToxCast_ACEA_T47D_80

# **BBB Prompt**

In [9]:
prompt_name = prompts[0]
prompt = tdc_prompts_json[prompt_name]
input_type = "{Drug SMILES}"

for drug, SMILE in drugs.items():
  TDC_PROMPT = prompt.replace(input_type, SMILE)

  outputs = pipe(TDC_PROMPT, max_new_tokens=8)
  response = outputs[0]["generated_text"]
  print(drug, "\n", response, "\n")

rivipansel 
 Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: C[C@H]1[C@H]([C@H]([C@@H]([C@@H](O1)O[C@@H]2[C@H](C[C@H](C[C@H]2O[C@H]3[C@@H]([C@H]([C@H]([C@H](O3)CO)O)O[C@@H](CC4CCCCC4)C(=O)O)OC(=O)C5=CC=CC=C5)C(=O)NCCNC(=O)COCCOCC(=O)NC6=C7C(=CC(=C6)S(=O)(=O)O)C=C(C=C7S(=O)(=O)O)S(=O)(=O)O)NC(=O)C8=CC(=O)NC(=O)N8)O)O)O
Answer:(A) 

senicapoc 
 Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the 

# **BindingDB_kd**
lower Kd = stronger binding

In [10]:
prompt_name = prompts[1]
prompt = tdc_prompts_json[prompt_name]
drug_type = "{Drug SMILES}"
aa_input = "{Target amino acid sequence}"


for drug, SMILE in drugs.items():
  drug_prompt = prompt.replace(drug_type, SMILE)
  for protein, amino_acids in proteins.items():
    TDC_PROMPT = drug_prompt.replace(aa_input, amino_acids)

    outputs = pipe(TDC_PROMPT, max_new_tokens=8)
    response = outputs[0]["generated_text"]
    print(drug, protein, "\n", response, "\n")

rivipansel E-selectin 
 Instructions: Answer the following question about drug target interactions.
Context: Drug-target binding is the physical interaction between a drug and a specific biological molecule, such as a protein or enzyme. This interaction is essential for the drug to exert its pharmacological effect. The strength of the drug-target binding is determined by the binding affinity, which is a measure of how tightly the drug binds to the target. Kd is the dissociation constant of a drug-target complex. It is the concentration of drug at which half of the drug-target complexes have dissociated. A lower Kd value indicates a stronger binding affinity.
Question: Given the target amino acid sequence and compound SMILES string, predict their normalized binding affinity Kd from 000 to 1000, where 000 is minimum Kd and 1000 is maximum Kd.
Drug SMILES: C[C@H]1[C@H]([C@H]([C@@H]([C@@H](O1)O[C@@H]2[C@H](C[C@H](C[C@H]2O[C@H]3[C@@H]([C@H]([C@H]([C@H](O3)CO)O)O[C@@H](CC4CCCCC4)C(=O)O)OC(=O

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


aspirin E-selectin 
 Instructions: Answer the following question about drug target interactions.
Context: Drug-target binding is the physical interaction between a drug and a specific biological molecule, such as a protein or enzyme. This interaction is essential for the drug to exert its pharmacological effect. The strength of the drug-target binding is determined by the binding affinity, which is a measure of how tightly the drug binds to the target. Kd is the dissociation constant of a drug-target complex. It is the concentration of drug at which half of the drug-target complexes have dissociated. A lower Kd value indicates a stronger binding affinity.
Question: Given the target amino acid sequence and compound SMILES string, predict their normalized binding affinity Kd from 000 to 1000, where 000 is minimum Kd and 1000 is maximum Kd.
Drug SMILES: CC(=O)OC1=CC=CC=C1C(=O)O
Target amino acid sequence: MIASQFLSALTLVLLIKESGAWSYNTSTEAMTYDEASAYCQQRYTHLVAIQNKEEIEYLNSILSYSPSYYWIGIRKVNNVWVWV

# **BindingDB_ic50**

In [11]:
prompt_name = prompts[2]
prompt = tdc_prompts_json[prompt_name]
drug_type = "{Drug SMILES}"
aa_input = "{Target amino acid sequence}"


for drug, SMILE in drugs.items():
  drug_prompt = prompt.replace(drug_type, SMILE)
  for protein, amino_acids in proteins.items():
    TDC_PROMPT = drug_prompt.replace(aa_input, amino_acids)

    outputs = pipe(TDC_PROMPT, max_new_tokens=8)
    response = outputs[0]["generated_text"]
    print(drug, protein, "\n", response, "\n")

rivipansel E-selectin 
 Instructions: Answer the following question about drug target interactions.
Context: Drug-target binding is the physical interaction between a drug and a specific biological molecule, such as a protein or enzyme. This interaction is essential for the drug to exert its pharmacological effect. The strength of the drug-target binding is determined by the binding affinity, which is a measure of how tightly the drug binds to the target. IC50 is the concentration of a drug that inhibits a biological activity by 50%. It is a measure of the drug's potency, but it is not a direct measure of binding affinity. 
Question: Given the target amino acid sequence and compound SMILES string, predict their normalized binding affinity Kd from 000 to 1000, where 000 is minimum IC50 and 1000 is maximum IC50.
Drug SMILES: C[C@H]1[C@H]([C@H]([C@@H]([C@@H](O1)O[C@@H]2[C@H](C[C@H](C[C@H]2O[C@H]3[C@@H]([C@H]([C@H]([C@H](O3)CO)O)O[C@@H](CC4CCCCC4)C(=O)O)OC(=O)C5=CC=CC=C5)C(=O)NCCNC(=O)COCC

# **BindingDB_Patent**

In [12]:
prompt_name = prompts[4]
prompt = tdc_prompts_json[prompt_name]
drug_type = "{Drug SMILES}"
aa_input = "{Target amino acid sequence}"


for drug, SMILE in drugs.items():
  drug_prompt = prompt.replace(drug_type, SMILE)
  for protein, amino_acids in proteins.items():
    TDC_PROMPT = drug_prompt.replace(aa_input, amino_acids)

    outputs = pipe(TDC_PROMPT, max_new_tokens=8)
    response = outputs[0]["generated_text"]
    print(drug, protein, "\n", response, "\n")

rivipansel E-selectin 
 Instructions: Answer the following question about drug target interactions.
Context: Drug-target interaction prediction task aims to predict the interaction activity score in silico given only the accessible compound structural information and protein amino acid sequence.
Question: Given the drug SMILES string and target amino acid sequence predict the normalized binding affinity from 000 to 1000, where 000 is minimum binding affinity and 1000 is maximum binding affinity.
Drug SMILES: C[C@H]1[C@H]([C@H]([C@@H]([C@@H](O1)O[C@@H]2[C@H](C[C@H](C[C@H]2O[C@H]3[C@@H]([C@H]([C@H]([C@H](O3)CO)O)O[C@@H](CC4CCCCC4)C(=O)O)OC(=O)C5=CC=CC=C5)C(=O)NCCNC(=O)COCCOCC(=O)NC6=C7C(=CC(=C6)S(=O)(=O)O)C=C(C=C7S(=O)(=O)O)S(=O)(=O)O)NC(=O)C8=CC(=O)NC(=O)N8)O)O)O
Target amino acid sequence: MIASQFLSALTLVLLIKESGAWSYNTSTEAMTYDEASAYCQQRYTHLVAIQNKEEIEYLNSILSYSPSYYWIGIRKVNNVWVWVGTQKPLTEEAKNWAPGEPNNRQKDEDCVEIYIKREKDVGMWNDERCSKKKLALCYTAACTNTSCSGHGECVETINNYTCKCDPGFSGLKCEQIVNCTALESPEHGSLVCSHPLGN

# **Custom prompts**

Trial definitions come from: [NIH Clinical trils info](https://www.nih.gov/health-information/nih-clinical-research-trials-you/basics)

In [13]:
custom_prompts = {
    "disgenet": "Instructions: Answer the following question about disease gene associations.\n"
    "Context: Many diseases are driven by genes aberrations. Gene-disease associations (GDA) quantify the relation among a pair of gene and disease. The GDA is usually constructed as a network where we can probe the gene-disease mechanisms by taking into account multiple genes and diseases factors. This task is to predict the association of any gene and disease from both a biochemical modeling and network edge classification perspectives.\n"
    "Question: Given the disease description and the amino acid of the gene, predict their association from 000 to 1000, where 000 is the minimum association and 1000 is the maximum association."
    "Disease: {Disease Description}\n"
    "Amino acid sequence: {Amino acid sequence}\n"
    "Answer:",

    "phase_1": "Instructions: Answer the following question about clinical trials.\n"
    "Context: Clinical trial is the most time and cost-consuming step in the drug discovery process. Phase 1 clinical trials test the safety and basic properties of a new drug or treatment in a small group of people for the first time. Optimizing and designing trials with machine learning could drastically lead to the speedup of delivery of life-saving therapeutics to patients. Clinical trial outcome prediction is a machine learning task that aims to forecast the outcome of clinical trials, such as the approval rate of a drug or treatment. It utilizes various clinical trial features, including the drug’s molecular structure and patient disease.\n"
    "Question: Given a drug SMILES string and disease, predict if the phase 1 trial (A) would not be approved (B) would be approved.\n"
    "Drug SMILES: {Drug SMILES}\n"
    "Disease: {Disease}\n"
    "Answer:",

    "phase_2": "Instructions: Answer the following question about clinical trials.\n"
    "Context: Clinical trial is the most time and cost-consuming step in the drug discovery process. Phase 2 clinical trials test the new drug or treatment in a larger group of people (100–300) to determine its effectiveness and to further study its safety. Optimizing and designing trials with machine learning could drastically lead to the speedup of delivery of life-saving therapeutics to patients. Clinical trial outcome prediction is a machine learning task that aims to forecast the outcome of clinical trials, such as the approval rate of a drug or treatment. It utilizes various clinical trial features, including the drug’s molecular structure and patient disease.\n"
    "Question: Given a drug SMILES string and disease, predict if the phase 2 trial (A) would not be approved (B) would be approved.\n"
    "Drug SMILES: {Drug SMILES}\n"
    "Disease: {Disease}\n"
    "Answer:",

    "phase_3": "Instructions: Answer the following question about clinical trials.\n"
    "Context: Clinical trial is the most time and cost-consuming step in the drug discovery process. Phase 3 clinical trials test the new drug or treatment in a large groups of people (1,000–3,000) to confirm its effectiveness, monitor side effects, compare it with standard or similar treatments, and collect information that will allow the new drug or treatment to be used safely. Optimizing and designing trials with machine learning could drastically lead to the speedup of delivery of life-saving therapeutics to patients. Clinical trial outcome prediction is a machine learning task that aims to forecast the outcome of clinical trials, such as the approval rate of a drug or treatment. It utilizes various clinical trial features, including the drug’s molecular structure and patient disease.\n"
    "Question: Given a drug SMILES string and disease, predict if the phase 3 trial (A) would not be approved (B) would be approved.\n"
    "Drug SMILES: {Drug SMILES}\n"
    "Disease: {Disease}\n"
    "Answer:"
}

# **DisGeNET**

In [14]:
prompt_name = "disgenet"
prompt = custom_prompts[prompt_name]
disease_type = "{Disease Description}"
aa_input = "{Amino acid sequence}"


for disease in diseases:
  disease_prompt = prompt.replace(disease_type, disease)
  for protein, amino_acids in proteins.items():
    TDC_PROMPT = disease_prompt.replace(aa_input, amino_acids)

    outputs = pipe(TDC_PROMPT, max_new_tokens=8)
    response = outputs[0]["generated_text"]
    print(disease, protein, "\n", response, "\n")

sickle cell disease E-selectin 
 Instructions: Answer the following question about disease gene associations.
Context: Many diseases are driven by genes aberrations. Gene-disease associations (GDA) quantify the relation among a pair of gene and disease. The GDA is usually constructed as a network where we can probe the gene-disease mechanisms by taking into account multiple genes and diseases factors. This task is to predict the association of any gene and disease from both a biochemical modeling and network edge classification perspectives.
Question: Given the disease description and the amino acid of the gene, predict their association from 000 to 1000, where 000 is the minimum association and 1000 is the maximum association.Disease: sickle cell disease
Amino acid sequence: MIASQFLSALTLVLLIKESGAWSYNTSTEAMTYDEASAYCQQRYTHLVAIQNKEEIEYLNSILSYSPSYYWIGIRKVNNVWVWVGTQKPLTEEAKNWAPGEPNNRQKDEDCVEIYIKREKDVGMWNDERCSKKKLALCYTAACTNTSCSGHGECVETINNYTCKCDPGFSGLKCEQIVNCTALESPEHGSLVCSHPLGNFSYNSSCSISCDRG

# **Phase I Trials**

In [15]:
prompt_name = "phase_1"
prompt = custom_prompts[prompt_name]
disease_type = "{Disease}"
drug_type = "{Drug SMILES}"


for disease in diseases:
  disease_prompt = prompt.replace(disease_type, disease)
  for drug, smile in drugs.items():
    TDC_PROMPT = disease_prompt.replace(drug_type, smile)

    outputs = pipe(TDC_PROMPT, max_new_tokens=8)
    response = outputs[0]["generated_text"]
    print(disease, "|", drug, "\n", response, "\n")

sickle cell disease | rivipansel 
 Instructions: Answer the following question about clinical trials.
Context: Clinical trial is the most time and cost-consuming step in the drug discovery process. Phase 1 clinical trials test the safety and basic properties of a new drug or treatment in a small group of people for the first time. Optimizing and designing trials with machine learning could drastically lead to the speedup of delivery of life-saving therapeutics to patients. Clinical trial outcome prediction is a machine learning task that aims to forecast the outcome of clinical trials, such as the approval rate of a drug or treatment. It utilizes various clinical trial features, including the drug’s molecular structure and patient disease.
Question: Given a drug SMILES string and disease, predict if the phase 1 trial (A) would not be approved (B) would be approved.
Drug SMILES: C[C@H]1[C@H]([C@H]([C@@H]([C@@H](O1)O[C@@H]2[C@H](C[C@H](C[C@H]2O[C@H]3[C@@H]([C@H]([C@H]([C@H](O3)CO)O)O[C@@

# **Phase II Trials**

In [16]:
prompt_name = "phase_2"
prompt = custom_prompts[prompt_name]
disease_type = "{Disease}"
drug_type = "{Drug SMILES}"


for disease in diseases:
  disease_prompt = prompt.replace(disease_type, disease)
  for drug, smile in drugs.items():
    TDC_PROMPT = disease_prompt.replace(drug_type, smile)

    outputs = pipe(TDC_PROMPT, max_new_tokens=8)
    response = outputs[0]["generated_text"]
    print(disease, "|", drug, "\n", response, "\n")

sickle cell disease | rivipansel 
 Instructions: Answer the following question about clinical trials.
Context: Clinical trial is the most time and cost-consuming step in the drug discovery process. Phase 2 clinical trials test the new drug or treatment in a larger group of people (100–300) to determine its effectiveness and to further study its safety. Optimizing and designing trials with machine learning could drastically lead to the speedup of delivery of life-saving therapeutics to patients. Clinical trial outcome prediction is a machine learning task that aims to forecast the outcome of clinical trials, such as the approval rate of a drug or treatment. It utilizes various clinical trial features, including the drug’s molecular structure and patient disease.
Question: Given a drug SMILES string and disease, predict if the phase 2 trial (A) would not be approved (B) would be approved.
Drug SMILES: C[C@H]1[C@H]([C@H]([C@@H]([C@@H](O1)O[C@@H]2[C@H](C[C@H](C[C@H]2O[C@H]3[C@@H]([C@H]([C@

# **Phase III Trials**

In [17]:
prompt_name = "phase_3"
prompt = custom_prompts[prompt_name]
disease_type = "{Disease}"
drug_type = "{Drug SMILES}"


for disease in diseases:
  disease_prompt = prompt.replace(disease_type, disease)
  for drug, smile in drugs.items():
    TDC_PROMPT = disease_prompt.replace(drug_type, smile)

    outputs = pipe(TDC_PROMPT, max_new_tokens=8)
    response = outputs[0]["generated_text"]
    print(disease, "|", drug, "\n", response, "\n")

sickle cell disease | rivipansel 
 Instructions: Answer the following question about clinical trials.
Context: Clinical trial is the most time and cost-consuming step in the drug discovery process. Phase 3 clinical trials test the new drug or treatment in a large groups of people (1,000–3,000) to confirm its effectiveness, monitor side effects, compare it with standard or similar treatments, and collect information that will allow the new drug or treatment to be used safely. Optimizing and designing trials with machine learning could drastically lead to the speedup of delivery of life-saving therapeutics to patients. Clinical trial outcome prediction is a machine learning task that aims to forecast the outcome of clinical trials, such as the approval rate of a drug or treatment. It utilizes various clinical trial features, including the drug’s molecular structure and patient disease.
Question: Given a drug SMILES string and disease, predict if the phase 3 trial (A) would not be approve

## Explore conversational capabilities with TxGemma-Chat

TxGemma features conversational models that add reasoning and explainability to predictions and can be used in multi-turn interactions. Their conversational ability comes at the expense of some predictive power.

**For this section, make sure that you have selected a TxGemma-Chat model variant.**

### Ask questions in a multi-turn conversation

This section demonstrates prompting TxGemma for conversational use, adhering to the [Gemma chat template](https://ai.google.dev/gemma/docs/core/prompt-structure).

In this example, first prompt the model to answer a question regarding a predictive task using the TDC format. Then, ask a follow-up question requesting the model to provide its reasoning for the predicted answer.

**Run the model directly**

In [ ]:
from IPython.display import display, Markdown

questions = [
    TDC_PROMPT,  # Initial question is a predictive task from TDC
    "Explain your reasoning based on the molecule structure."
]

messages = []

display(Markdown("\n\n---\n\n"))
for question in questions:
    display(Markdown(f"**User:**\n\n{question}\n\n---\n\n"))
    messages.append(
        { "role": "user", "content": question },
    )
    # Apply the tokenizer's built-in chat template
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens=512)
    response = tokenizer.decode(outputs[0, len(inputs[0]):], skip_special_tokens=True)
    display(Markdown(f"**TxGemma:**\n\n{response}\n\n---\n\n"))
    messages.append(
        { "role": "assistant", "content": response},
    )



---



**User:**

Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer:

---



**TxGemma:**

(B)

---



**User:**

Explain your reasoning based on the molecule structure.

---



**TxGemma:**

Here's the breakdown of why the drug with SMILES CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21 likely crosses the BBB:

* **Small Size:** The molecule is relatively small, which is a general favorable characteristic for BBB penetration. Larger molecules have a harder time squeezing through the tight junctions between brain endothelial cells.
* **Lipophilicity (Hydrophobicity):** The presence of multiple carbon and hydrogen atoms in the benzene rings and alkyl chains makes the molecule predominantly lipophilic (fat-loving).  Lipophilicity is a key determinant of BBB permeability. The more lipophilic a drug, the easier it crosses the lipid-rich cell membranes of the BBB.
* **Lack of Charged Groups:**  The molecule lacks large, charged groups (like carboxyl or amine groups).  Charged groups tend to be repelled by the lipid bilayer of the BBB, making it harder to cross. 
* **Absence of Specific BBB Targets:** While some drugs have specific transporters that help them across the BBB, this molecule doesn't appear to have any obvious targeting groups.

**In summary, the drug's small size, lipophilic nature, lack of significant charge, and absence of specific BBB-targeting groups suggest that it likely possesses a good ability to cross the blood-brain barrier.**

**Important Note:** This is a general prediction based on structural features. Actual BBB permeability is complex and can be influenced by many factors. Experimental validation is always necessary to confirm drug penetration ability. 


---



**Run with the `pipeline` API**

In [ ]:
from IPython.display import display, Markdown

questions = [
    TDC_PROMPT,  # Initial question is a predictive task from TDC
    "Explain your reasoning based on the molecule structure."
]

messages = []

display(Markdown("\n\n---\n\n"))
for question in questions:
    display(Markdown(f"**User:**\n\n{question}\n\n---\n\n"))
    messages.append(
        { "role": "user", "content": question },
    )
    outputs = pipe(messages, max_new_tokens=512)
    messages = outputs[0]["generated_text"]
    response = messages[-1]["content"].strip()
    display(Markdown(f"**TxGemma:**\n\n{response}\n\n---\n\n"))



---



**User:**

Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer:

---



**TxGemma:**

(B)

---



**User:**

Explain your reasoning based on the molecule structure.

---



**TxGemma:**

Here's the breakdown of why the drug with SMILES CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21 likely crosses the BBB:

* **Small Size:** The molecule is relatively small, which is a general favorable characteristic for BBB penetration. Larger molecules have a harder time squeezing through the tight junctions between brain endothelial cells.
* **Lipophilicity (Hydrophobicity):** The presence of multiple carbon and hydrogen atoms in the benzene rings and alkyl chains makes the molecule predominantly lipophilic (fat-loving).  Lipophilicity is a key determinant of BBB permeability. The more lipophilic a drug, the easier it crosses the lipid-rich cell membranes of the BBB.
* **Lack of Charged Groups:**  The molecule lacks large, charged groups (like carboxyl or amine groups).  Charged groups tend to be repelled by the lipid bilayer of the BBB, making it harder to cross. 
* **Absence of Specific BBB Targets:** While some drugs have specific transporters that help them across the BBB, this molecule doesn't appear to have any obvious targeting groups.

**In summary, the drug's small size, lipophilic nature, lack of significant charge, and absence of specific BBB-targeting groups suggest that it likely possesses a good ability to cross the blood-brain barrier.**

**Important Note:** This is a general prediction based on structural features. Actual BBB permeability is complex and can be influenced by many factors. Experimental validation is always necessary to confirm drug penetration ability.

---



# Next steps

Explore the other [notebooks](https://github.com/google-gemini/gemma-cookbook/blob/main/TxGemma) to learn what else you can do with the model.